In [ ]:
#안건들려도 됨
!pip install music21
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install midi2audio
!pip install pyFluidSynth
!pip install pretty_midi

  Preparing metadata (setup.py) ... done
  Created wheel for pyFluidSynth: filename=pyFluidSynth-1.3.3-py3-none-any.whl size=19185 sha256=428fb4c69d5640dbf8a317af2e4da3664f072056e596288028ce35ad2dc445a4
  Stored in directory: /root/.cache/pip/wheels/e6/9c/2b/b4b194cbb100d6f20136a22b2f76cd329b1a21139ed26c9775
Successfully built pyFluidSynth
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=388c6e77e664c1fc443dcd004aa8b96cb95b61f6ab16f5d7e12dce69d3ddc736
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1

In [ ]:
#안건드려도 됨
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#음악 데이터의 노트 데이터를 불러오는 작업
import os
import numpy as np
from music21 import converter, instrument, note, chord

# 데이터셋 경로와 모델 이름을 입력 받기
dataset_name = input("Enter the dataset name (e.g., mozart_dataset, radwimps_dataset, classic_dataset, beethoven_dataset, maestro_dataset, liszt_dataset, LaCampanella_dataset): ")
model_name = input("Enter the model name (e.g., mozart_model, radwimps_model, classic_model, beethoven_model, maestro_model, liszt_model, LaCampanella_model): ")

# MIDI 파일 경로
midi_folder = f'/content/drive/MyDrive/new_dateset/{dataset_name}'
midi_files = [os.path.join(midi_folder, f) for f in os.listdir(midi_folder) if f.endswith('.mid')]

notes = []

for file in midi_files:
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

# 노트 데이터 확인
print(f"Total notes extracted: {len(notes)}")

Enter the dataset name (e.g., mozart_dataset, radwimps_dataset, classic_dataset, beethoven_dataset, maestro_dataset, liszt_dataset, LaCampanella_dataset): beethoven_dataset
Enter the model name (e.g., mozart_model, radwimps_model, classic_model, beethoven_model, maestro_model, liszt_model, LaCampanella_model): beethoven_model
Total notes extracted: 5053


In [ ]:
# 노트 데이터 전처리(인공지능이 학습가능한 데이터로 재가공)
sequence_length = 100
n_vocab = len(set(notes))

# 노트를 정수로 매핑
note_to_int = {note: number for number, note in enumerate(sorted(set(notes)))}

# 입력 시퀀스와 출력 노트 생성
network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# 데이터 정규화
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = np.eye(n_vocab)[network_output]

print(f"Total patterns: {n_patterns}")

Total patterns: 4953


In [ ]:
#음악 파일을 생성하기위한 AI 모델 형성 및 불러오기
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras.optimizers import Adam

# 현재 데이터셋 기반으로 모델 재구축
model = Sequential()
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(n_vocab)) # 현재 데이터셋의 n_vocab 사용
model.add(Activation('softmax'))

optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


model.compile(loss='categorical_crossentropy', optimizer='adam')

# 모델 요약 출력
model.summary()
model_path = f'/content/drive/MyDrive/new_dateset/{model_name}.weights.h5' # Change file extension

if os.path.exists(model_path):
    # 가중치 파일이 있으면 불러오기
    model.load_weights(model_path)
    print(f"Loaded weights from {model_path}")
else:
    # 가중치 파일이 없으면 생성 후 저장
    print(f"Weight file not found. Creating a new model and saving weights to {model_path}")

    model.save_weights(model_path)

model.load_weights(model_path)

# 최종 모델 저장
model.save(f'/content/drive/MyDrive/new_dateset/{model_name}_final.h5') # You can keep this as .h5

Model: "sequential_66"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_198 (LSTM)                      │ (None, 100, 512)            │       1,052,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_132 (Dropout)                │ (None, 100, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_199 (LSTM)                      │ (None, 100, 512)            │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_133 (Dropout)                │ (None, 100, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_200 (LSTM)                      │ (None, 512)                 │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_66 (Dense)                     │ (None, 259)                 │         132,867 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_66 (Activation)           │ (None, 259)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,383,939 (20.54 MB)

 Trainable params: 5,383,939 (20.54 MB)

 Non-trainable params: 0 (0.00 B)

Loaded weights from /content/drive/MyDrive/new_dateset/beethoven_model.weights.h5


In [ ]:
#위에 학습 코드(가중치 불러오기)
#필수

from tensorflow.keras.models import load_model

# 모델 이름 입력 받기
model_name = input("Enter the model name to load weights from (e.g., mozart_model, radwimps_model, classic_model, beethoven_model, maestro_model, liszt_model, LaCampanella_model): ")

model.load_weights(f'/content/drive/MyDrive/new_dateset/{model_name}.keras')


model.save(f'/content/drive/MyDrive/new_dateset/{model_name}_final.keras')

Enter the model name to load weights from (e.g., mozart_model, radwimps_model, classic_model, beethoven_model, maestro_model, liszt_model, LaCampanella_model): beethoven_model


In [ ]:
#음악 생성 코드
import random
from music21 import stream, note, chord, instrument, converter
import numpy as np

# 노트와 인덱스를 상호 변환할 사전 정의
int_to_note = {number: note for note, number in note_to_int.items()}

# 시드 시퀀스 준비
start = np.random.randint(0, len(network_input) - 1)
pattern = network_input[start]
prediction_output = []

# 500개의 노트를 생성
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]

# 초반 예측 노트 100개 버리기
prediction_output = prediction_output[50:]

# 노트를 MIDI 파일로 변환
offset = 0
output_notes = []

# 사용자가 선택한 악기 입력받기
instrument_name = input("사용할 악기를 입력하세요 (Piano, ElectricPiano, Organ, Accordion, Harmonica, Violin, Violoncello, Contrabass, Harp, Guitar, ElectricGuitar, AcousticBass, ElectricBass, Recorder, PanFlute, Clarinet, Saxophone): ")

# 생성된 노트를 음악21 객체로 변환
for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(getattr(instrument,instrument_name)())  # 입력된 악기 사용
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        output_notes.append(getattr(instrument,instrument_name)())  # 입력된 악기 사용
        output_notes.append(new_note)
    offset += 0.4   # 정상 템포=0.5, 빠른 템포=0.35

# 새로운 MIDI 파일 생성
music_name = input("파일 이름을 적어주세요: ")
midi_stream = stream.Stream(output_notes)
midi_fp = f'/content/drive/MyDrive/new_dateset/download_file/{music_name}.mid'
midi_stream.write('midi', fp=midi_fp)

from google.colab import files
files.download(midi_fp)



사용할 악기를 입력하세요 (Piano, ElectricPiano, Organ, Accordion, Harmonica, Violin, Violoncello, Contrabass, Harp, Guitar, ElectricGuitar, AcousticBass, ElectricBass, Recorder, PanFlute, Clarinet, Saxophone): ElectricPiano
파일 이름을 적어주세요: y


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os

# 데이터셋 이름 입력 받기
dataset_name = input("Enter the dataset name (e.g., mozart_dataset, radwimps_dataset): ")

# 데이터셋 경로 설정
midi_folder = f'/content/drive/MyDrive/new_dateset/{dataset_name}'

# 경로 확인
if os.path.exists(midi_folder):
    print("Directory exists")
else:
    print("Directory does not exist")

# 파일 목록 확인
for root, dirs, files in os.walk(midi_folder):
    for file in files:
        print(file)

Enter the dataset name (e.g., mozart_dataset, radwimps_dataset): Trout_dataset
Directory exists
Trout Quintet (Forellenquintett) - 4th Movement_ Andantino - Franz Schubert.mid
[Free-scores.com]_schubert-franz-peter-die-forelle-4th-movement-6843.midi
Trout Quintet (Forellenquintett) - 4th Movement_ Andantino - Franz Schubert (1).mid


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# 훈련 손실 값 그래프 그리기
plt.plot(history.epoch, history.history['loss'], label='Train Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim(0, 6)  # y축 범위 설정
plt.legend()
plt.show()


NameError: name 'history' is not defined

In [ ]:
###건들지 마시요 중요!!!1



from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras.optimizers import Adam
import os


# 모델 파일 경로
model_path = f'/content/drive/MyDrive/new_dateset/{model_name}.keras'

# 모델이 존재하는지 확인
if os.path.exists(model_path):
    model.load_weights(model_path)
else:
    print(f"No model found at {model_path}. A new model will be trained and saved.")

# 체크포인트 생성
checkpoint = ModelCheckpoint(
    model_path,
    monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

# 모델 훈련 및 저장
history = model.fit(network_input, network_output, epochs=50, batch_size=64, callbacks=callbacks_list)

# 최종 모델 저장
model.save(model_path)

Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 4.5839
Epoch 1: loss improved from inf to 4.56296, saving model to /content/drive/MyDrive/new_dateset/Trout_model.keras
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 4.5831
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 4.5682
Epoch 2: loss did not improve from 4.56296
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 4.5681
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 4.5286
Epoch 3: loss did not improve from 4.56296
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 4.5299
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 4.5017
Epoch 4: loss did not improve from 4.56296
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 4.5039
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 4.5451
Epoch 5: loss improved from 4.56296 to 4.56223, saving model to /content/drive/MyDrive/new_dateset/Trout_model.keras
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 4.5457
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 